In [1]:
# Cell 1: Import libraries and load data
import pandas as pd
import sqlite3  
import time

# Load expanded RFM data
rfm_df = pd.read_csv("../Data/Processed/rfm_table_expanded.csv")

print("=" * 60)
print("PHASE 2: SQL ANALYSIS")
print("=" * 60)
print(f"Loaded {len(rfm_df):,} customers from expanded dataset")
print(f"\nColumns: {list(rfm_df.columns)}")
print("\nFirst 5 rows:")
print(rfm_df.head())


PHASE 2: SQL ANALYSIS
Loaded 10,500 customers from expanded dataset

Columns: ['CustomerID', 'Recency', 'Frequency', 'Monetary']

First 5 rows:
   CustomerID  Recency  Frequency  Monetary
0       12346      326          1  77183.60
1       12347        2          7   4310.00
2       12348       75          4   1797.24
3       12349       19          1   1757.55
4       12350      310          1    334.40


In [2]:
# Cell 2: Create SQLite database from your CSV
print("=" * 60)
print("CREATING SQL DATABASE...")
print("=" * 60)

# Create a connection to a new database file
# If the file doesn't exist, SQLite creates it automatically
conn = sqlite3.connect("../Data/Processed/rfm_database.db")

# Write your pandas DataFrame to the database as a table called 'customers'
rfm_df.to_sql('customers', conn, if_exists='replace', index=False)

print(" Database created: rfm_database.db")
print(" Table created: 'customers'")
print(f" Rows inserted: {len(rfm_df):,}")

# Don't close connection yet - we'll use it in next cells


CREATING SQL DATABASE...
 Database created: rfm_database.db
 Table created: 'customers'
 Rows inserted: 10,500


In [3]:
# Cell 3: Your first SQL query - Select everything
print("=" * 60)
print("QUERY 1: SELECT ALL DATA (First 10 rows)")
print("=" * 60)

# Write SQL query as a string
query = """
SELECT * 
FROM customers 
LIMIT 10;
"""

# Execute the query and convert results to DataFrame
result = pd.read_sql_query(query, conn)

print("SQL Query:")
print(query)
print("\nResult:")
print(result)


QUERY 1: SELECT ALL DATA (First 10 rows)
SQL Query:

SELECT * 
FROM customers 
LIMIT 10;


Result:
   CustomerID  Recency  Frequency  Monetary
0       12346      326          1  77183.60
1       12347        2          7   4310.00
2       12348       75          4   1797.24
3       12349       19          1   1757.55
4       12350      310          1    334.40
5       12352       36          8   2506.04
6       12353      204          1     89.00
7       12354      232          1   1079.40
8       12355      214          1    459.40
9       12356       23          3   2811.43


In [4]:
# Cell 4: Filter customers with Recency < 30 days (recent shoppers)
print("=" * 60)
print("QUERY 2: FIND RECENT CUSTOMERS (Recency < 30 days)")
print("=" * 60)

query = """
SELECT CustomerID, Recency, Frequency, Monetary
FROM customers
WHERE Recency < 30
ORDER BY Recency ASC
LIMIT 10;
"""

result = pd.read_sql_query(query, conn)

print("SQL Query:")
print(query)
print(f"\nFound {len(result)} recent customers")
print("\nFirst 10:")
print(result)


QUERY 2: FIND RECENT CUSTOMERS (Recency < 30 days)
SQL Query:

SELECT CustomerID, Recency, Frequency, Monetary
FROM customers
WHERE Recency < 30
ORDER BY Recency ASC
LIMIT 10;


Found 10 recent customers

First 10:
   CustomerID  Recency  Frequency  Monetary
0       12423        1          8   1859.31
1       12433        1          7  13375.87
2       12518        1          5   2056.89
3       12526        1          3   1316.66
4       12662        1         11   3849.78
5       12680        1          4    862.81
6       12713        1          1    848.55
7       12748        1        209  33719.73
8       12955        1         11   4757.16
9       12985        1          2   1239.38


In [6]:
# Cell 5: Grouping customers into recency buckets
print("=" * 60)
print("QUERY 5: CUSTOMERS BY RECENCY GROUPS")
print("=" * 60)

query = """
SELECT 
    CASE 
        WHEN Recency <= 30 THEN 'Very Recent (0-30 days)'
        WHEN Recency <= 90 THEN 'Recent (31-90 days)'
        WHEN Recency <= 180 THEN 'At Risk (91-180 days)'
        ELSE 'Hibernating (180+ days)'
    END AS RecencyGroup,
    COUNT(*) AS CustomerCount,
    ROUND(AVG(Frequency), 1) AS AvgFrequency,
    ROUND(AVG(Monetary), 2) AS AvgSpend,
    ROUND(SUM(Monetary), 2) AS TotalRevenue
FROM customers
GROUP BY RecencyGroup
ORDER BY 
    CASE RecencyGroup
        WHEN 'Very Recent (0-30 days)' THEN 1
        WHEN 'Recent (31-90 days)' THEN 2
        WHEN 'At Risk (91-180 days)' THEN 3
        ELSE 4
    END;
"""

result = pd.read_sql_query(query, conn)

print(" SQL Query:")
print(query)
print("\n Results:")
print(result)


QUERY 5: CUSTOMERS BY RECENCY GROUPS
 SQL Query:

SELECT 
    CASE 
        WHEN Recency <= 30 THEN 'Very Recent (0-30 days)'
        WHEN Recency <= 90 THEN 'Recent (31-90 days)'
        WHEN Recency <= 180 THEN 'At Risk (91-180 days)'
        ELSE 'Hibernating (180+ days)'
    END AS RecencyGroup,
    COUNT(*) AS CustomerCount,
    ROUND(AVG(Frequency), 1) AS AvgFrequency,
    ROUND(AVG(Monetary), 2) AS AvgSpend,
    ROUND(SUM(Monetary), 2) AS TotalRevenue
FROM customers
GROUP BY RecencyGroup
ORDER BY 
    CASE RecencyGroup
        WHEN 'Very Recent (0-30 days)' THEN 1
        WHEN 'Recent (31-90 days)' THEN 2
        WHEN 'At Risk (91-180 days)' THEN 3
        ELSE 4
    END;


 Results:
              RecencyGroup  CustomerCount  AvgFrequency  AvgSpend  \
0  Very Recent (0-30 days)           3995           7.5   4948.51   
1      Recent (31-90 days)           2455           3.0   1392.39   
2    At Risk (91-180 days)           2115           2.0   1017.86   
3  Hibernating (180+ day

In [7]:
# Cell 6: Finding the top 10 highest spending customers
print("=" * 60)
print("QUERY 6: TOP 10 VIP CUSTOMERS")
print("=" * 60)

query = """
SELECT 
    CustomerID,
    Recency AS DaysSinceLastPurchase,
    Frequency AS TotalOrders,
    ROUND(Monetary, 2) AS TotalSpent,
    ROUND(Monetary / Frequency, 2) AS AvgOrderValue
FROM customers
ORDER BY Monetary DESC
LIMIT 10;
"""

result = pd.read_sql_query(query, conn)

print(" SQL Query:")
print(query)
print("\n Top 10 Customers:")
print(result)


QUERY 6: TOP 10 VIP CUSTOMERS
 SQL Query:

SELECT 
    CustomerID,
    Recency AS DaysSinceLastPurchase,
    Frequency AS TotalOrders,
    ROUND(Monetary, 2) AS TotalSpent,
    ROUND(Monetary / Frequency, 2) AS AvgOrderValue
FROM customers
ORDER BY Monetary DESC
LIMIT 10;


 Top 10 Customers:
   CustomerID  DaysSinceLastPurchase  TotalOrders  TotalSpent  AvgOrderValue
0       14646                      2           73   280206.02        3838.44
1       18102                      1           60   259657.30        4327.62
2       17450                      8           46   194550.79        4229.36
3       16446                      1            2   168472.50       84236.25
4       14911                      1          201   143825.06         715.55
5       12415                     24           21   124914.53        5948.31
6       14156                     10           55   117379.63        2134.18
7       17511                      3           31    91062.38        2937.50
8       16029

In [9]:
# Cell 7: Calculate summary statistics for all RFM metrics
print("=" * 60)
print("QUERY 7: RFM SUMMARY STATISTICS")
print("=" * 60)

query = """
SELECT 
    COUNT(*) AS TotalCustomers,
    ROUND(AVG(Recency), 1) AS AvgRecency,
    MIN(Recency) AS MinRecency,
    MAX(Recency) AS MaxRecency,
    ROUND(AVG(Frequency), 1) AS AvgFrequency,
    MIN(Frequency) AS MinFrequency,
    MAX(Frequency) AS MaxFrequency,
    ROUND(AVG(Monetary), 2) AS AvgMonetary,
    ROUND(MIN(Monetary), 2) AS MinMonetary,
    ROUND(MAX(Monetary), 2) AS MaxMonetary,
    ROUND(SUM(Monetary), 2) AS TotalRevenue
FROM customers;
"""

result = pd.read_sql_query(query, conn)

print(" SQL Query:")
print(query)
print("\n Overall Statistics:")
print(result)


QUERY 7: RFM SUMMARY STATISTICS
 SQL Query:

SELECT 
    COUNT(*) AS TotalCustomers,
    ROUND(AVG(Recency), 1) AS AvgRecency,
    MIN(Recency) AS MinRecency,
    MAX(Recency) AS MaxRecency,
    ROUND(AVG(Frequency), 1) AS AvgFrequency,
    MIN(Frequency) AS MinFrequency,
    MAX(Frequency) AS MaxFrequency,
    ROUND(AVG(Monetary), 2) AS AvgMonetary,
    ROUND(MIN(Monetary), 2) AS MinMonetary,
    ROUND(MAX(Monetary), 2) AS MaxMonetary,
    ROUND(SUM(Monetary), 2) AS TotalRevenue
FROM customers;


 Overall Statistics:
   TotalCustomers  AvgRecency  MinRecency  MaxRecency  AvgFrequency  \
0           10500        91.8           1         388           4.2   

   MinFrequency  MaxFrequency  AvgMonetary  MinMonetary  MaxMonetary  \
0             1           209      2505.15         3.75    280206.02   

   TotalRevenue  
0   26304076.08  


In [11]:
# Cell 8: Compare SQL vs Pandas speed
print("=" * 60)
print("SPEED TEST: SQL vs PANDAS")
print("=" * 60)

# Test SQL
start_time = time.time()
sql_query = "SELECT * FROM customers WHERE Recency < 50;"
sql_result = pd.read_sql_query(sql_query, conn)
sql_time = time.time() - start_time

# Test Pandas
start_time = time.time()
pandas_result = rfm_df[rfm_df['Recency'] < 50]
pandas_time = time.time() - start_time

# Show results
print(f"  SQL time:    {sql_time:.5f} seconds")
print(f"  Pandas time: {pandas_time:.5f} seconds")
print(f"\n SQL found {len(sql_result):,} customers")
print(f" Pandas found {len(pandas_result):,} customers")

if sql_time < pandas_time:
    print(f"\n SQL was {pandas_time/sql_time:.2f}x faster!")
else:
    print(f"\n Pandas was {sql_time/pandas_time:.2f}x faster!")


SPEED TEST: SQL vs PANDAS
  SQL time:    0.04538 seconds
  Pandas time: 0.00125 seconds

 SQL found 5,026 customers
 Pandas found 5,026 customers

 Pandas was 36.29x faster!


In [12]:
# Cell 9: RFM segment assignment using SQL
print("=" * 60)
print("QUERY 9: SEGMENT ASSIGNMENT")
print("=" * 60)

# NTILE(5) divides customers into 5 equal groups (quintiles)
# We use it to score R, F, M from 1-5

query = """
SELECT 
    CustomerID,
    Recency,
    Frequency, 
    Monetary,
    NTILE(5) OVER (ORDER BY Recency DESC) AS R_Score,
    NTILE(5) OVER (ORDER BY Frequency ASC) AS F_Score,
    NTILE(5) OVER (ORDER BY Monetary ASC) AS M_Score,
    CASE
        WHEN NTILE(5) OVER (ORDER BY Recency DESC) >= 4 
         AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 4
         AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 4
        THEN 'Champions'
        
        WHEN NTILE(5) OVER (ORDER BY Frequency ASC) >= 3
         AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 3
        THEN 'Loyal Customers'
        
        WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 2
         AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 2
        THEN 'At Risk'
        
        WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 1
         AND NTILE(5) OVER (ORDER BY Frequency ASC) <= 2
        THEN 'Hibernating'
        
        ELSE 'Others'
    END AS Segment
FROM customers
LIMIT 20;
"""

result = pd.read_sql_query(query, conn)

print(" SQL Query:")
print(query)
print("\n First 20 Customers with Segments:")
print(result)


QUERY 9: SEGMENT ASSIGNMENT
 SQL Query:

SELECT 
    CustomerID,
    Recency,
    Frequency, 
    Monetary,
    NTILE(5) OVER (ORDER BY Recency DESC) AS R_Score,
    NTILE(5) OVER (ORDER BY Frequency ASC) AS F_Score,
    NTILE(5) OVER (ORDER BY Monetary ASC) AS M_Score,
    CASE
        WHEN NTILE(5) OVER (ORDER BY Recency DESC) >= 4 
         AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 4
         AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 4
        THEN 'Champions'

        WHEN NTILE(5) OVER (ORDER BY Frequency ASC) >= 3
         AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 3
        THEN 'Loyal Customers'

        WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 2
         AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 2
        THEN 'At Risk'

        WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 1
         AND NTILE(5) OVER (ORDER BY Frequency ASC) <= 2
        THEN 'Hibernating'

        ELSE 'Others'
    END AS Segment
FROM customers
LIMIT 20;


 First 20 Customers with Segments

In [14]:
# Cell 10: Count how many customers in each segment
print("=" * 60)
print("QUERY 10: SEGMENT DISTRIBUTION")
print("=" * 60)

query = """
SELECT 
    Segment,
    COUNT(*) AS CustomerCount,
    ROUND(AVG(Monetary), 2) AS AvgSpending,
    ROUND(SUM(Monetary), 2) AS TotalRevenue,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customers), 1) AS PercentOfTotal
FROM (
    SELECT 
        CustomerID,
        Monetary,
        CASE
            WHEN NTILE(5) OVER (ORDER BY Recency DESC) >= 4 
             AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 4
             AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 4
            THEN 'Champions'
            
            WHEN NTILE(5) OVER (ORDER BY Frequency ASC) >= 3
             AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 3
            THEN 'Loyal Customers'
            
            WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 2
             AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 2
            THEN 'At Risk'
            
            WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 1
             AND NTILE(5) OVER (ORDER BY Frequency ASC) <= 2
            THEN 'Hibernating'
            
            ELSE 'Others'
        END AS Segment
    FROM customers
) AS segmented_customers
GROUP BY Segment
ORDER BY CustomerCount DESC;
"""

result = pd.read_sql_query(query, conn)

print(" SQL Query:")
print(query)
print("\n Segment Summary:")
print(result)


QUERY 10: SEGMENT DISTRIBUTION
 SQL Query:

SELECT 
    Segment,
    COUNT(*) AS CustomerCount,
    ROUND(AVG(Monetary), 2) AS AvgSpending,
    ROUND(SUM(Monetary), 2) AS TotalRevenue,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM customers), 1) AS PercentOfTotal
FROM (
    SELECT 
        CustomerID,
        Monetary,
        CASE
            WHEN NTILE(5) OVER (ORDER BY Recency DESC) >= 4 
             AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 4
             AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 4
            THEN 'Champions'

            WHEN NTILE(5) OVER (ORDER BY Frequency ASC) >= 3
             AND NTILE(5) OVER (ORDER BY Monetary ASC) >= 3
            THEN 'Loyal Customers'

            WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 2
             AND NTILE(5) OVER (ORDER BY Frequency ASC) >= 2
            THEN 'At Risk'

            WHEN NTILE(5) OVER (ORDER BY Recency DESC) <= 1
             AND NTILE(5) OVER (ORDER BY Frequency ASC) <= 2
            THEN 'Hibe

In [16]:
# Cell 11: Clean up - close database connection
print("=" * 60)
print("CLOSING DATABASE")
print("=" * 60)

conn.close()

print(" Connection closed")
print("\n Phase 2 Complete!")
print("\n You now know:")
print("  ✓ Basic SELECT, WHERE, ORDER BY")
print("  ✓ Aggregations (COUNT, AVG, SUM)")
print("  ✓ GROUP BY for summaries")
print("  ✓ CASE WHEN for conditions")
print("  ✓ NTILE for scoring")
print("  ✓ Subqueries")
print("\n Next: Phase 3 - Tableau Dashboards!")


CLOSING DATABASE
 Connection closed

 Phase 2 Complete!

 You now know:
  ✓ Basic SELECT, WHERE, ORDER BY
  ✓ Aggregations (COUNT, AVG, SUM)
  ✓ GROUP BY for summaries
  ✓ CASE WHEN for conditions
  ✓ NTILE for scoring
  ✓ Subqueries

 Next: Phase 3 - Tableau Dashboards!
